In [3]:
from bs4 import BeautifulSoup
import requests
import re

In [4]:
def get_numbers(lst):
    new_list = [int(x.split(':')[1]) if ':' in x else -1 for x in lst]
    return new_list

class RecipeFetcher:

    search_base_url = 'https://www.allrecipes.com/search/results/?wt=%s&sort=re'

    def search_recipes(self, keywords): 
        search_url = self.search_base_url %(keywords.replace(' ','+'))

        page_html = requests.get(search_url)
        page_graph = BeautifulSoup(page_html.content)

        return [recipe.a['href'] for recipe in\
               page_graph.find_all('div', {'class':'grid-card-image-container'})]

    def scrape_recipe(self, recipe_url):
        results = {}

        page_html = requests.get(recipe_url)
        print(recipe_url)
        page_graph = BeautifulSoup(page_html.content)

        results['ingredients'] = [ingredient.text for ingredient in\
                                  page_graph.find_all('span', {'itemprop':'recipeIngredient'})]

        results['directions'] = [direction.text.strip() for direction in\
                                 page_graph.find_all('span', {'class':'recipe-directions__list--item'})
                                 if direction.text.strip()]

        results['nutrition'] = self.scrape_nutrition_facts(recipe_url)
        
        results['calories_and_servings'] = self.scrape_calories_servings(recipe_url)

        return results
    
    def scrape_nutrition_facts(self, recipe_url):
        results = []

        nutrition_facts_url = '%s/fullrecipenutrition' %(recipe_url)

        page_html = requests.get(nutrition_facts_url)
        page_graph = BeautifulSoup(page_html.content)

        r = re.compile("([0-9]*\.?[0-9]*)([a-zA-Z]+)")
        
        nutrient = {}

        for nutrient_row in page_graph.find_all('div', {'class': 'nutrition-row'}):
            
            lst = nutrient_row.text.split(':')
            amount_lst = lst[1]
            name = lst[0].replace('\n', '')
            
            amount = amount_lst.split('\n')
            amount = [x.replace(' ', '') for x in amount[:2]]
            
            nutrient[name] = amount
            
        return nutrient
    
    def scrape_calories_servings(self, recipe_url):
        """
        returns [servings per recipe, amt per serving, calories]
        """

        nutrition_facts_url = '%s/fullrecipenutrition' %(recipe_url)

        page_html = requests.get(nutrition_facts_url)
        page_graph = BeautifulSoup(page_html.content)

        r = re.compile("([0-9]*\.?[0-9]*)([a-zA-Z]+)")
        
        nutrient = {}

        for row in page_graph.find_all('div', {'class': 'nutrition-top light-underline'}):
            lst = row.text.split('\n')
            lst = list(filter(lambda a: a != '\r', lst))
            
            calories = [x.lstrip() for x in lst]
            calories.pop()
            info = get_numbers(calories)
            
            return info
      

In [128]:
rf = RecipeFetcher()
url = rf.search_recipes('vegetable soup')[0]
res = rf.scrape_recipe(url)

https://www.allrecipes.com/recipe/13028/italian-vegetable-soup/


In [131]:
ingredients_list = res['ingredients']
print('ORIGINAL INGREDIENTS:')
print(ingredients_list)
print()

new_ingredients = multiply_recipe(ingredients_list, multiplier=2)
print('NEW INGREDIENTS:')
print(new_ingredients)

ORIGINAL INGREDIENTS:
['1 pound ground beef', '1 cup chopped onion', '1 cup chopped celery', '1 cup chopped carrots', '2 cloves garlic, minced', '1 (14.5 ounce) can peeled and diced tomatoes', '1 (15 ounce) can tomato sauce', '2 (19 ounce) cans kidney beans, drained and rinsed', '2 cups water', '5 teaspoons beef bouillon granules', '1 tablespoon dried parsley', '1/2 teaspoon dried oregano', '1/2 teaspoon dried basil', '2 cups chopped cabbage', '1 (15.25 ounce) can whole kernel corn', '1 (15 ounce) can green beans', '1 cup macaroni']

NEW INGREDIENTS:
['2  pounds ground beef', '2  cups chopped onion', '2  cups chopped celery', '2  cups chopped carrots', '4  cloves garlic, minced', '2 (14.5 ounce) can peeled and diced tomatoes', '2 (15 ounce) can tomato sauce', '4  (19 ounce) cans kidney beans, drained and rinsed', '4  cups water', '10  teaspoons beef bouillon granules', '2  tablespoons dried parsley', '1  teaspoon dried oregano', '1  teaspoon dried basil', '4  cups chopped cabbage', '2 

In [1]:
MEASUREMENT_WORDS = ['cup', 'oz', 'ounce', 'pound', 'clove', 'teaspoon', 'tsp', 'tablespoon', 'tbs', 'egg', 'jar', 'package', 'pack', 'can']

def multiply_recipe(ingredients_list, multiplier):
    
    new_ingredients_list = []
    
    # go through each ingredient in the recipe
    for ingredient in ingredients_list:
        tokens = ingredient.split()
        original_quantity = 0
        ingredient_name = ''
        
        # split ingredient into tokens to separate the 'quantity' from 'ingredient_name'
        for token in tokens:            
            if any([str(digit) in token for digit in range(10)]) and not any([char in token for char in ['(', ')']]):
                fraction_obj = sum(map(fractions.Fraction, token.split()))
                as_float = float(fraction_obj)
                original_quantity += as_float
            else:
                ingredient_name = ingredient_name + ' ' + token

        # multiply amount by 'multiplier'
        new_quantity = original_quantity * multiplier
        
        # check if 'new_quantity' crosses 1
        # append/delete plural 's' from end of measurement word
        # Ex: 3 cups --> 1 cup, 1/2 tablespoon --> 2 tablespoons
        if original_quantity <= 1 and new_quantity > 1:
            tokens = ingredient_name.split()
            if tokens[0] in MEASUREMENT_WORDS:
                tokens[0] = ' '+ tokens[0] + 's'
            ingredient_name = ' '.join(tokens)
        elif original_quantity > 1 and new_quantity <= 1:
            tokens = ingredient_name.split()
            if tokens[0][0:-1] in MEASUREMENT_WORDS:
                tokens[0] = tokens[0][0:-1]
            ingredient_name = ' '.join(tokens)
        
        # convert float to mixed fraction
        base = int(new_quantity // 1)
        if base == new_quantity:
            new_num = str(base) + ' '
        elif new_quantity < 1:
            frac = fractions.Fraction.from_float(new_quantity).limit_denominator()
            new_num = str(frac)
        else:
            frac = fractions.Fraction.from_float(new_quantity % base).limit_denominator()
            new_num = str(base) + ' ' + str(frac)
        
        # update the multiplied quantity in the ingredients list
        if new_quantity != 0:
            new_ingredients_list.append(new_num + ingredient_name)
        else:
            new_ingredients_list.append(ingredient_name[1:])
          
    return new_ingredients_list